# Customer churn prediction with deep learning (ANN)
@author: Talardia Gbangou, October 2022

Customer churn prediction is to measure why customers are leaving a business. Precision, recall, f1-score are used to measure performance of our model

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

LOAD THE DATA

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


DROP UNNECCESSARY COLUMNS

In [4]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


FEATURE ENGINEERING

In [5]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [6]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [7]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


ONE HOT ENCODING CATEGORICAL VALUES

In [8]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [9]:
print_unique_col_values(df1)

In [10]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


SCALING THE DATASET

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


SPLITTING THE DATASET INTO TRAINING AND TEST SET

In [13]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)

In [15]:
print(X_train.shape)
print(X_test.shape)

(8000, 13)
(2000, 13)


IMPORTING TENSORFLOW LIBRARIES

In [16]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

BUILD THE MODEL(ANN)

In [17]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [18]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

2022-10-29 00:00:51.590963: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 00:00:55.982949: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.5005 - accuracy: 0.7885
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4718 - accuracy: 0.7970
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4594 - accuracy: 0.8010
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4465 - accuracy: 0.8071
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4332 - accuracy: 0.8148
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4207 - accuracy: 0.8174
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4094 - accuracy: 0.8242
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3972 - accuracy: 0.8306
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3855 - accuracy: 0.8372
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3742 - accura

250/250 [==============================] - 1s 4ms/step - loss: 0.3330 - accuracy: 0.8635
Epoch 82/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3344 - accuracy: 0.8644
Epoch 83/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3336 - accuracy: 0.8637
Epoch 84/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3332 - accuracy: 0.8637
Epoch 85/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3329 - accuracy: 0.8633
Epoch 86/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3332 - accuracy: 0.8635
Epoch 87/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3336 - accuracy: 0.8636
Epoch 88/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3327 - accuracy: 0.8658
Epoch 89/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3328 - accuracy: 0.8645
Epoch 90/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3332 - accuracy: 

As We See, the precision, recall and f1 score of Class 1 is very low due to imbalanced dataset

Mitigating Skewdness of Data

Method 1: Undersampling

reference: https: // www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

In [19]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [20]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [21]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [22]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [23]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 5ms/step - loss: 0.6894 - accuracy: 0.5539
Epoch 2/100
102/102 [==============================] - 1s 5ms/step - loss: 0.6751 - accuracy: 0.5750
Epoch 3/100
102/102 [==============================] - 1s 5ms/step - loss: 0.6601 - accuracy: 0.5913
Epoch 4/100
102/102 [==============================] - 0s 4ms/step - loss: 0.6447 - accuracy: 0.6306
Epoch 5/100
102/102 [==============================] - 1s 5ms/step - loss: 0.6303 - accuracy: 0.6499
Epoch 6/100
102/102 [==============================] - 1s 9ms/step - loss: 0.6190 - accuracy: 0.6658
Epoch 7/100
102/102 [==============================] - 1s 7ms/step - loss: 0.6067 - accuracy: 0.6790
Epoch 8/100
102/102 [==============================] - 1s 7ms/step - loss: 0.5951 - accuracy: 0.6889
Epoch 9/100
102/102 [==============================] - 1s 8ms/step - loss: 0.5855 - accuracy: 0.6971
Epoch 10/100
102/102 [==============================] - 1s 6ms/step - loss: 0.5765 - accura

102/102 [==============================] - 1s 7ms/step - loss: 0.4620 - accuracy: 0.7739
Epoch 82/100
102/102 [==============================] - 1s 6ms/step - loss: 0.4599 - accuracy: 0.7775
Epoch 83/100
102/102 [==============================] - 0s 4ms/step - loss: 0.4599 - accuracy: 0.7739
Epoch 84/100
102/102 [==============================] - 1s 5ms/step - loss: 0.4600 - accuracy: 0.7745
Epoch 85/100
102/102 [==============================] - 0s 5ms/step - loss: 0.4609 - accuracy: 0.7757
Epoch 86/100
102/102 [==============================] - 1s 6ms/step - loss: 0.4607 - accuracy: 0.7763
Epoch 87/100
102/102 [==============================] - 1s 6ms/step - loss: 0.4607 - accuracy: 0.7705
Epoch 88/100
102/102 [==============================] - 1s 6ms/step - loss: 0.4590 - accuracy: 0.7782
Epoch 89/100
102/102 [==============================] - 0s 4ms/step - loss: 0.4610 - accuracy: 0.7782
Epoch 90/100
102/102 [==============================] - 1s 6ms/step - loss: 0.4594 - accuracy: 

As we see, there is considerable Improvement in the f1, recall and precision scores of Class 1 Value. The f1 score has improved from 0.58 to 0.77.

Method2: Oversampling

In [25]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [26]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [27]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 3s 6ms/step - loss: 0.6504 - accuracy: 0.6254
Epoch 2/100
399/399 [==============================] - 2s 5ms/step - loss: 0.6110 - accuracy: 0.6669
Epoch 3/100
399/399 [==============================] - 2s 6ms/step - loss: 0.5853 - accuracy: 0.6926
Epoch 4/100
399/399 [==============================] - 2s 4ms/step - loss: 0.5555 - accuracy: 0.7162
Epoch 5/100
399/399 [==============================] - 2s 6ms/step - loss: 0.5321 - accuracy: 0.7370
Epoch 6/100
399/399 [==============================] - 2s 6ms/step - loss: 0.5143 - accuracy: 0.7482
Epoch 7/100
399/399 [==============================] - 2s 5ms/step - loss: 0.5017 - accuracy: 0.7543
Epoch 8/100
399/399 [==============================] - 2s 4ms/step - loss: 0.4931 - accuracy: 0.7586
Epoch 9/100
399/399 [==============================] - 2s 4ms/step - loss: 0.4870 - accuracy: 0.7622
Epoch 10/100
399/399 [==============================] - 2s 4ms/step - loss: 0.4843 - accura

399/399 [==============================] - 2s 5ms/step - loss: 0.4491 - accuracy: 0.7849
Epoch 82/100
399/399 [==============================] - 2s 4ms/step - loss: 0.4499 - accuracy: 0.7825
Epoch 83/100
399/399 [==============================] - 2s 5ms/step - loss: 0.4486 - accuracy: 0.7833
Epoch 84/100
399/399 [==============================] - 2s 5ms/step - loss: 0.4488 - accuracy: 0.7834
Epoch 85/100
399/399 [==============================] - 2s 4ms/step - loss: 0.4491 - accuracy: 0.7843
Epoch 86/100
399/399 [==============================] - 2s 4ms/step - loss: 0.4493 - accuracy: 0.7836
Epoch 87/100
399/399 [==============================] - 2s 5ms/step - loss: 0.4480 - accuracy: 0.7834
Epoch 88/100
399/399 [==============================] - 2s 5ms/step - loss: 0.4483 - accuracy: 0.7832
Epoch 89/100
399/399 [==============================] - 2s 5ms/step - loss: 0.4486 - accuracy: 0.7857
Epoch 90/100
399/399 [==============================] - 2s 5ms/step - loss: 0.4487 - accuracy: 